In [2]:
'''Copyright by the Massachusetts Institute of Technology.  All rights reserved'''
import matplotlib as mpl
mpl.use('Agg')
# import matplotlib.pyplot as plt,pandas as pd
# from os import makedirs
import numpy as np, sys, h5py, argparse, subprocess
from os.path import join,dirname,basename,exists
from os import system
import os
import tensorflow as tf
from keras.models import model_from_json
from keras import backend as K
from tensorflow.keras import models
import json
import pdb
import keras
# os.environ['KERAS_BACKEND'] = 'theano'
keras.backend.set_image_data_format('channels_first')

In [3]:
IsAnneal=True
avoidnc=True
buff_range=10
# arch='best_archit.json'
# weight='bestmodel_weights.h5'
constraint='l2'
amino=np.asarray(['I', 'L', 'V', 'F', 'M', 'C', 'A', 'G', 'P', 'T', 'S', 'Y', 'W', 'Q', 'N', 'H', 'E', 'D', 'K', 'R', 'X'])
L2coeff = 0.1

def mat2oh(datain,trim1,):
	orig_seq=[]
	trims=np.bool_(trim1)
	onehot=datain.reshape(datain.shape[0],datain.shape[1],datain.shape[3])
	colrank=np.argsort(onehot, axis=1)
	colind=np.argmax(onehot, axis=1)
	for i in range(colind.shape[0]):
		temp = colind[i][trims[i]]
		temp[onehot[i][0][trims[i]]==0.05]=20
		sequence=''.join(amino[temp])
		orig_seq.append(sequence)
	orig_seq=np.asarray(orig_seq)
	return orig_seq


def iterate(input_tensor, model):
	with tf.GradientTape() as gtape:
		gtape.watch(input_tensor)
		score = model(input_tensor)  # score.shape == (2000, 1)
		if constraint == 'l2':
			final_score = score - L2coeff * K.sum((input_tensor ** 2))
		else:
			final_score = score
		grads = gtape.gradient(final_score, input_tensor)
	
	return grads, score, final_score


def iterate2(input_tensor, model):
	
	score = model(input_tensor)
	if constraint == 'l2':
		final_score = score - L2coeff * K.sum((input_tensor ** 2))
	else:
		final_score = score
	
	return score, final_score

In [4]:
BASE_DIR2SAVE = '/home/jhbyun/ResearchProjects/DrugDesign/antibody-2019/checkpoint'  # local path
BASE_RESULTDIR = '/home/jhbyun/ResearchProjects/DrugDesign/antibody-2019/result'  # local path
BASE_SEEDDIR = '/home/jhbyun/ResearchProjects/DrugDesign/antibody-2019/seed_seq'  # local path
    
class Args:
    rootpath = BASE_DIR2SAVE
    architecture = 'seq_32x1_16_filt3'
    resultdir = BASE_RESULTDIR
    iteration = 70
    stepsize = 5e-03
    seed_dir = BASE_SEEDDIR
    task_type = 'reg'
    saveh5 = 0

args = Args()

In [5]:
base = args.rootpath
arc = args.architecture
architecture_file = join(base, arc + '.txt')
weight_file = join(base, arc + '.h5')
resultdir = join(args.resultdir, arc)

iteration = args.iteration
step = args.stepsize
os.makedirs(resultdir, exist_ok=True)
inputdir = args.seed_dir
task_type = args.task_type

# what the filename should include:
# model architecture
# iteration
# step

tmp_folder = f"{arc}"

savename=join(resultdir,'input_gen'+str(iteration)+'-'+str(step))
outname=join(resultdir,'genseq-'+str(iteration)+'-'+str(step)+'.tsv')

#------parsing model layers----------
with open(architecture_file, "r") as f:
    json_file = json.load(f)

model = model_from_json(json_file)
del json_file

model.load_weights(weight_file)

layer_input = model.layers[0].input
layer_size=model.layers[0].input_shape
print("input size:", layer_size)
if task_type=='reg':
    output_layer = model.layers[-1].output
elif task_type=='class':
    output_layer = model.layers[-2].output
#---------create h5py file----------
if args.saveh5:
    f = h5py.File(savename,'w')
    grp =f.create_group("inputgen")
    grp2=f.create_group("track")
#--------parse input file-----------
input_all=np.asarray([]).reshape((0,layer_size[1],layer_size[2],layer_size[3]))

if os.path.isdir(inputdir):
    files = os.listdir(inputdir)
    files = [os.path.join(inputdir, f) for f in files]
    files = [files[0],]  # delete this afterwards
else:
    files = [inputdir,]

for batchfile in files:
    fi = h5py.File(batchfile, 'r')
    dataset = np.asarray(fi['data'])
    label=np.asarray(fi['label'])
    label=label[:,0]
    input_all=np.append(input_all,dataset,axis=0)

if args.saveh5:
    grp.create_dataset("orig_seq",data=input_all.reshape(input_all.shape[0],input_all.shape[1],input_all.shape[3]))

input size: (None, 20, 1, 20)


In [6]:
input_all.shape

(5000, 20, 1, 20)

In [7]:
input_data = input_all[0: 2000]


In [10]:
input_data[0].shape

(20, 1, 20)

In [9]:
mat2oh(input_data[0], )

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0

In [ ]:
best_map=np.asarray([]).reshape((0,layer_size[1],layer_size[2],layer_size[3]))
record_all=np.asarray([]).reshape((0,layer_size[1],layer_size[2],layer_size[3]))
record_seed=np.asarray([])#seed for each proposed sequence
# below not used
# oh_map=np.asarray([]).reshape((0,layer_size[1],layer_size[2],layer_size[3]))
# oh_act=np.asarray([])
oh_seq=np.asarray([])
# final_act=np.asarray([])
record_act=np.asarray([])
record_sact=np.asarray([])
best_act=np.asarray([])
init_act=np.asarray([])
init_loss=np.asarray([])
best_it = np.asarray([])
orig_seq_list = np.asarray([])
# convg_it = np.asarray([])

for batch in range(0,input_all.shape[0],2000):
    input_data=input_all[batch:min(batch+2000,input_all.shape[0])]
    datain=np.copy(input_data)
    tryi=input_data.shape[0]
    input_tensor = tf.convert_to_tensor(input_data, dtype=tf.float32)

    best_input =np.zeros(input_data.shape)
    record_all_seq=np.asarray([]).reshape(0,datain.shape[1],datain.shape[2],datain.shape[3])
    record_all_act=np.asarray([])
    record_all_orig=np.asarray([])
    record_seed_act=np.asarray([])
    convg_input=np.zeros(input_data.shape)
    best_activation = np.asarray([-1.0]*tryi)
    best_loss = np.asarray([-1000000000.0]*tryi)
    convg_activation = np.asarray([-10000.0]*tryi)
    best_iter = np.asarray([-1]*tryi)
    convg_iter=np.asarray([-1]*tryi)
    loss_track = []
    activation_track = []
    count=0

    activation_init, loss_init=iterate2(input_tensor, model)
    activation_init, loss_init = activation_init.numpy(), loss_init.numpy()

    init_act=np.append(init_act,activation_init[:,0],axis=0)
    init_loss=np.append(init_loss,loss_init[:,0],axis=0)
    holdcnt = np.asarray([0]*tryi)
    lr=step
    print('')
    print('')
    print('---------------------------------------------------------------')
    print('Initial Average Enrichment Score = ', np.mean(activation_init))
    print('---------------------------------------------------------------')
    print('')
    print('')
    mask=np.array([False for i in range(tryi)])
    trim1=np.sum(datain.reshape(datain.shape[0],datain.shape[1],datain.shape[3]),axis=1)!=0
    # trim1.shape == (2000,20)

    trim2=np.tile(~trim1,(1,20)).reshape(input_data.shape[0],input_data.shape[1],input_data.shape[2],input_data.shape[3])
    # trim2.shape == (2000, 20, 1, 20)

    record_trim=np.asarray([]).reshape(0,trim1.shape[1])
    orig_seq=mat2oh(datain, trim1)

    orig_seq_list = np.append(orig_seq_list, orig_seq, axis=0)
    while True:
        for grow in range(iteration):
            input_tensor = tf.convert_to_tensor(input_data, dtype=tf.float32)
            grads_value, _, _ = iterate(input_tensor, model)
            grads_value = grads_value.numpy()
            # what is mask?
            grads_value[mask,:,:,:]=0
            if count>100:
                lr=max(step*(count-100)**(-0.2),1e-6)
                # lr = step
            # Probably this part is the gradient ascent
            input_data += grads_value*lr
            input_data[trim2] = 0.0
            pdb.set_trace()

        activation_all, loss_all = iterate2(input_tensor, model)
        activation_all, loss_all = activation_all.numpy(), loss_all.numpy()

        print('---------------------------------------------------')
        print('Iteration', count*iteration)
        print('Average Enrichment Score (Float Vector)', np.mean(activation_all))
        print('Number of sequences Converged', sum(mask))
        print(float(sum(mask))*100./2000, " % of sequences converged")
        onehot=input_data.reshape(input_data.shape[0],input_data.shape[1],input_data.shape[3])
        colrank=np.argsort(onehot,axis=1)
        colind=np.argmax(onehot,axis=1)
        oh=np.zeros(onehot.shape)

        for x in range(onehot.shape[0]):
            for y in range(onehot.shape[2]):
                if avoidnc:
                    if amino[colind[x,y]]=='N' or amino[colind[x,y]]=='C':
                        #print 'N detected, changing to second large'
                        colind[x,y]=colrank[x,-2,y]
                        oh[x,colind[x,y],y]=1.0

        oh=oh.reshape(oh.shape[0],layer_size[1],layer_size[2],layer_size[3])
        oh[trim2]=0.0
        oh_activation, oh_loss = iterate2(oh.astype('float32'), model)
        oh_activation, oh_loss = oh_activation.numpy().reshape(-1), oh_loss.numpy().reshape(-1)

        print('Average Enrichment Score (One Hot Vector)', np.mean(oh_activation))
        print('---------------------------------------------------')
        activation_track.append(oh_activation)
        temp_act=np.copy(oh_activation)
        temp_act[mask]=-10000.0
        # reshape to ensure that temp_act and best_activation have identical dimensions
        improve=(temp_act.reshape(-1)>best_activation.reshape(-1))

        if sum(improve)>0:
            best_activation[improve] = oh_activation[improve]
            best_input[improve,:,:,:] = oh[improve,:,:,:]
            record_all_orig=np.append(record_all_orig,orig_seq[improve],axis=0)
            activation_init = activation_init.reshape(-1)
            record_seed_act=np.append(record_seed_act,activation_init[improve],axis=0)
            record_all_seq=np.append(record_all_seq,oh[improve,:,:,:],axis=0)
            record_all_act=np.append(record_all_act,oh_activation[improve],axis=0)
            record_trim=np.append(record_trim,trim1[improve,:],axis=0)
            best_iter[improve]=count
        holdcnt[improve]=0
        holdcnt[~improve]=holdcnt[~improve]+1
        mask=(holdcnt>=buff_range)  # what's this?
        if sum(mask)==tryi or count>1000:
            print('Converge at',count)
            print('Activation',np.mean(activation_all))
            print('Converged',sum(mask))
            print('Loss',np.mean(loss_all))
            print('Best score',np.mean(best_activation))
            out_seq=mat2oh(record_all_seq,record_trim)
            oh_seq=np.append(oh_seq,out_seq,axis=0)
            break

        count+=1

    best_map=np.append(best_map,best_input,axis=0)
    best_act=np.append(best_act,best_activation,axis=0)
    best_it=np.append(best_it,best_iter,axis=0)
    record_all=np.append(record_all,record_all_seq,axis=0)
    record_act=np.append(record_act,record_all_act,axis=0)
    record_seed=np.append(record_seed,record_all_orig,axis=0)
    record_sact=np.append(record_sact,record_seed_act,axis=0)

if args.saveh5:

    grp.create_dataset("best_seq",data=best_map)
    grp.create_dataset("onehot_matrix",data=record_all)

    encoding = lambda x: x.encode('utf-8')
    orig_seq_list = np.array(list(map(encoding, orig_seq_list)))
    oh_seq = np.array(list(map(encoding, oh_seq)))
    record_seed = np.array(list(map(encoding, record_seed)))

    grp2.create_dataset("init_act", data=init_act)
    grp2.create_dataset("init_loss", data=init_loss)
    grp2.create_dataset("original_sequence", data=orig_seq_list)  # *** TypeError: No conversion path for dtype: dtype('<U18')
    grp2.create_dataset("onehot_sequence", data=oh_seq)  # *** TypeError: No conversion path for dtype: dtype('<U32')
    grp2.create_dataset("seed_sequence",data=record_seed)  # *** TypeError: No conversion path for dtype: dtype('<U32')
    grp2.create_dataset("best_activation", data=best_act)
    grp2.create_dataset("all_activation", data=record_act)
    grp2.create_dataset("seed_activation", data=record_sact)
    f.close()

tsv=np.concatenate((np.arange(oh_seq.shape[0]).reshape(oh_seq.shape[0],1),
                    oh_seq.reshape(oh_seq.shape[0],1),
                    record_seed.reshape(record_seed.shape[0],1),
                    record_act.reshape(record_act.shape[0],1),
                    record_sact.reshape(record_sact.shape[0],1)),axis=1)

np.savetxt(outname,tsv,fmt='%s',delimiter='\t')

input size: (None, 20, 1, 20)


---------------------------------------------------------------
Initial Average Enrichment Score =  -0.6943287
---------------------------------------------------------------


> <ipython-input-4-2ac384e1862d>(128)<module>()
    126     orig_seq_list = np.append(orig_seq_list, orig_seq, axis=0)
    127     while True:
--> 128         for grow in range(iteration):
    129             input_tensor = tf.convert_to_tensor(input_data, dtype=tf.float32)
    130             grads_value, _, _ = iterate(input_tensor, model)

ipdb> input_data
array([[[[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        ...,

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0., ...,